In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from PIL import Image
import numpy as np
import cv2

In [135]:
im_file = "data/page_01.jpg"
image = cv2.imread(im_file)

In [112]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
print("Converted to grayscale image.")
cv2.imshow("img",gray_image)
cv2.waitKey(0)

Converted to grayscale image.


-1

In [52]:
thresh=cv2.threshold(gray_image,200,230,cv2.THRESH_BINARY)[1]
print("Threshold Applied sucessfully.")
cv2.imshow("img",thresh)
cv2.waitKey(0)

Threshold Applied sucessfully.


-1

In [5]:

def thick_font(image):
    image = cv2.bitwise_not(thresh)
    kernel = np.ones((1,2),np.uint8)
    image = cv2.dilate(image,kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image
    


In [6]:
dilated_image =thick_font(thresh)
cv2.imshow("img",dilated_image)
cv2.waitKey(0)

-1

In [7]:
decodec_text = pytesseract.image_to_string(dilated_image)
decodec_text

"The Life and Work of\nFredson Bowers\n\nby\nG. THOMAS TANSELLE\n\n'M EVERY FIELD OF ENDEAVOR THERE ARE 4 FEW FIGURES WHOSE ACCOM-\nplishment and influence cause them to be the symbols of their age;\ntheir careers and ocuvres become the touchstones by which the\nfield is measured and its history told. In the related pursuits of\n\nanalytical and descriptive bibliography, textual criticism, and scholarly\nediting, Fredson Bowers was such 2 figure, dominating the four decades\nafter 1949, when his Principles of Bibliographical Description was pub-\nlished. By 197g the period was already being called “the age of Bowers”:\nin that year Norman Sanders, writing the chapter on textual scholarship\nfor Stanley Wells's Shakespeare: Select Bibliographies, gave this title to\na section of his essay. For most people, it would be achievement enough\nto rise to such 2 position in a field as complex as Shakespearean textual\nstudies; but Bowers played an equally important role in other areas.\nEditor

In [10]:
lines = dict()
i=0
w=''
for line in decodec_text:
    w = w+line
    if line == "\n":
        lines[i]=w
        i=i+1
        w=''

In [11]:
lines

{0: 'The Life and Work of\n',
 1: 'Fredson Bowers\n',
 2: '\n',
 3: 'by\n',
 4: 'G. THOMAS TANSELLE\n',
 5: '\n',
 6: "'M EVERY FIELD OF ENDEAVOR THERE ARE 4 FEW FIGURES WHOSE ACCOM-\n",
 7: 'plishment and influence cause them to be the symbols of their age;\n',
 8: 'their careers and ocuvres become the touchstones by which the\n',
 9: 'field is measured and its history told. In the related pursuits of\n',
 10: '\n',
 11: 'analytical and descriptive bibliography, textual criticism, and scholarly\n',
 12: 'editing, Fredson Bowers was such 2 figure, dominating the four decades\n',
 13: 'after 1949, when his Principles of Bibliographical Description was pub-\n',
 14: 'lished. By 197g the period was already being called “the age of Bowers”:\n',
 15: 'in that year Norman Sanders, writing the chapter on textual scholarship\n',
 16: "for Stanley Wells's Shakespeare: Select Bibliographies, gave this title to\n",
 17: 'a section of his essay. For most people, it would be achievement enough\n',


In [12]:
context = {**lines}
print(context)


{0: 'The Life and Work of\n', 1: 'Fredson Bowers\n', 2: '\n', 3: 'by\n', 4: 'G. THOMAS TANSELLE\n', 5: '\n', 6: "'M EVERY FIELD OF ENDEAVOR THERE ARE 4 FEW FIGURES WHOSE ACCOM-\n", 7: 'plishment and influence cause them to be the symbols of their age;\n', 8: 'their careers and ocuvres become the touchstones by which the\n', 9: 'field is measured and its history told. In the related pursuits of\n', 10: '\n', 11: 'analytical and descriptive bibliography, textual criticism, and scholarly\n', 12: 'editing, Fredson Bowers was such 2 figure, dominating the four decades\n', 13: 'after 1949, when his Principles of Bibliographical Description was pub-\n', 14: 'lished. By 197g the period was already being called “the age of Bowers”:\n', 15: 'in that year Norman Sanders, writing the chapter on textual scholarship\n', 16: "for Stanley Wells's Shakespeare: Select Bibliographies, gave this title to\n", 17: 'a section of his essay. For most people, it would be achievement enough\n', 18: 'to rise to s

In [3]:
im_file = "data/sign.jpg"
image = cv2.imread(im_file)
im_h, im_w, im_d = image.shape
print(image.shape)
base_image = image.copy()

(716, 2500, 3)


In [4]:
gray =cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Apply blur to gray image
blur = cv2.GaussianBlur(gray,(7,7),0)

# Convert blur image to thresh image(Binary image)
thresh = cv2.threshold(blur,210,230,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Now we have to create a kernel object
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,13))

# Now apply dilation on the kernel
dilate = cv2.dilate(thresh,kernel,iterations=1)
cv2.imwrite("temp/OCR_dilate.png",dilate)

True

In [5]:
cv2.imshow("Original",dilate)
cv2.waitKey(0)

-1

In [6]:
gray =cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray,210,230,cv2.THRESH_BINARY)[1]

In [7]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [8]:
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if  h>200 and w >20:
        roi = thresh[y:y+h, x:x+w]
        cv2.rectangle(image,(x,y),(x+w,y+h),(36,255,12),2)
        ocr_result = pytesseract.image_to_string(roi)
        # ocr_result = ocr_result.split("\n")
        print(ocr_result)

cv2.imwrite("temp/OCR_box.png",image)

True

In [9]:
image = cv2.imread(im_file)

In [11]:
text = pytesseract.image_to_string(image)

In [12]:
text

''